In [ ]:
# Copyright 2025 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### ADK Agent Engine 배포

In [31]:
import os
from dotenv import load_dotenv
import argparse

import vertexai
from vertexai import agent_engines
from vertexai.agent_engines import AdkApp

from agent import root_agent

In [32]:
# 환경 변수 로드.
load_dotenv()

from dotenv import dotenv_values
config = dotenv_values("../../.env")
for key, value in config.items():
    print(f"{key}: {value}")

GOOGLE_GENAI_USE_VERTEXAI: TRUE
GOOGLE_CLOUD_PROJECT: ai-hangsik
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_GENAI_MODEL: gemini-2.5-flash
GOOGLE_GENAI_LIVE_MODEL: gemini-live-2.5-flash
AGENT_ENGINE_BUCKET: gs://agent_engine_0120
AGENTOPS_API_KEY: 43150e85-1c93-4c65-8526-3ac50cf1bcfc


#### Build ADK Agent.

In [33]:
# Vertex AI 환경 초기화 : Agent Engine을 배포하기 위한 환경 설정
# Agent Engine 과 Gemini 모델을 사용하기 위한 리전을 다르게 할 수 있음.
vertexai.init(
    project=os.getenv("GOOGLE_CLOUD_PROJECT"),
    location=os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket=os.getenv("AGENT_ENGINE_BUCKET"),
)

In [34]:
agent_name = "engine_0122"

In [35]:
# adk_app 생성
adk_app = AdkApp(agent=root_agent)

# Agent Engine 환경 설정
display_name = agent_name
gcs_dir_name = os.getenv("AGENT_ENGINE_BUCKET")
description = "사용자 질문에 대한 AI 정보 검색 어시스턴트"

requirements = [
    "google-adk[vertexai]",
    "google-cloud-aiplatform[adk,agent-engines]",
    "cloudpickle==3.0",
    "python-dotenv",
]

#### Deploy or Update an Agent Engine with ADK Framework.

In [36]:
# Trace 와 Logging 설정 
env_vars = {
  "GOOGLE_CLOUD_AGENT_ENGINE_ENABLE_TELEMETRY": "true",
  "OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT": "true",
}

create = False

if create:
    # 최초 Agent Engine을 디플로이.
    remote_agent = agent_engines.create(
                    adk_app,
                    display_name=display_name,
                    gcs_dir_name = gcs_dir_name,
                    description=description,
                    requirements=requirements,
                    extra_packages = []
    )
else:
    # 이미 디플로이된 Agent를 업데이트.
    # 아래 Resource name은 Console 에서 확인 가능.
    agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"
    
    remote_agent = agent_engines.update(
        resource_name = agent_engine_resource_name,
        agent_engine = adk_app
    )    

I0000 00:00:1768984134.244367 1241092 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Using bucket agent_engine_0120
Wrote to gs://agent_engine_0120/agent_engine/agent_engine.pkl
Update Agent Engine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880/operations/5749275091082936320
Agent Engine updated. Resource name: projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880


#### Run Agent with Python Code

In [37]:

agent_engine_resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"

remote_agent = agent_engines.get(agent_engine_resource_name)
print(f"Display name : {remote_agent.display_name}")
print(f"Reource name : {remote_agent.resource_name}")


remote_session = await remote_agent.async_create_session(user_id="u_456")
print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
    print("\n👤 User :   ")

    # 사용자 입력에서 쿼리 가져오기
    query = input("")
    if query.strip().lower() in ["exit", "quit"]:
        break
    else:
        print(query)

    events = remote_agent.async_stream_query(user_id="u_456", 
                                    session_id=remote_session["id"],
                                    message=query)
    
    async for event in events:
        response = event['content']['parts'][0]['text']
        print(f"\n 🤖 AI Assistant: {response}\n")


Display name : engine_0122
Reource name : projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880
AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.

👤 User :   
생성형 AI 란 ?

 🤖 AI Assistant: 생성형 AI(Generative AI)는 기존 데이터를 학습하여 새로운 콘텐츠를 만들어낼 수 있는 인공지능 모델을 의미합니다. 이는 단순한 데이터 분석이나 예측을 넘어, 학습된 데이터를 기반으로 독창적인 결과물을 '생성'하는 데 특화되어 있습니다.

주요 특징과 기능은 다음과 같습니다.
*   **데이터 학습 및 패턴 인식**: 생성형 AI는 대량의 텍스트, 이미지, 오디오, 비디오 등 다양한 데이터를 학습하여 그 속에 숨겨진 패턴, 스타일, 구조를 파악합니다.
*   **새로운 콘텐츠 생성**: 학습된 패턴을 바탕으로 기존에 없던 새로운 텍스트(글쓰기, 번역, 요약), 이미지(그림, 사진), 오디오(음악, 음성), 비디오 등을 생성할 수 있습니다.
*   **다양한 응용 분야**: 예술 창작, 디자인, 소프트웨어 코드 작성, 스토리텔링, 개인화된 추천, 신약 개발 등 광범위한 분야에서 활용되고 있습니다.
*   **대표적인 예시**: 텍스트 생성을 위한 GPT(Generative Pre-trained Transformer) 시리즈, 이미지 생성을 위한 DALL-E, Midjourney, Stable Diffusion 등이 있습니다.

요약하자면, 생성형 AI는 기존 데이터를 모방하고 재조합하는 것을 넘어, 창의적으로 새로운 결과물을 만들어내는 인공지능 기술이라고 할 수 있습니다.


👤 User :   
부정적인 측면은 ? 

 🤖 AI Assistant: 생성형 AI는 많은 잠재력을 가지고 있지만, 동시에 여러 가지 부정적인 측면과 위험을 내포하고 있습니다. 주요 문제점은 다음과 같습니다.

**

#### Run Agent with REST API(Python)

In [38]:
import requests
import google.auth
import google.auth.transport.requests

credentials, project = google.auth.default()
auth_request = google.auth.transport.requests.Request()
credentials.refresh(auth_request)
token = credentials.token

region = os.getenv("GOOGLE_CLOUD_LOCATION")

resource_name = "projects/721521243942/locations/us-central1/reasoningEngines/449705753318522880"
url = f"https://{region}-aiplatform.googleapis.com/v1/{resource_name}:streamQuery?alt=sse"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

print(f"AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.")

while True:
  print("\n👤 User :   ")

  # 사용자 입력에서 쿼리 가져오기
  query = input("")
  if query.strip().lower() in ["exit", "quit"]:
      break
  else:
      print(query)

  payload = {
      "class_method": "async_stream_query",
      "input": {
        "user_id": "u_456",
        "message": query,
      }
    }

  response = requests.post(url, json=payload, headers=headers)
  outcome = response.json()['content']['parts'][0]['text']

  print("\n 🤖 AI Assistant: ", outcome)

I0000 00:00:1768984402.167576 1241092 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


AI Agent를 종료하려면 'exit' 또는 'quit'을 입력하세요.

👤 User :   
생성형 AI 란 ?

 🤖 AI Assistant:  '생성형 AI'는 사용자의 요청에 따라 텍스트, 이미지, 비디오, 음악, 소프트웨어 코드와 같은 새롭고 독창적인 콘텐츠를 생성할 수 있는 인공지능 기술을 의미합니다.

**주요 특징 및 작동 원리:**
*   **새로운 콘텐츠 생성:** 단순히 기존 데이터를 분석하거나 정보를 제공하는 것을 넘어, 학습된 지식을 바탕으로 완전히 새로운 콘텐츠를 창조합니다.
*   **딥러닝 기반:** 방대한 양의 데이터를 학습하여 그 속에 있는 패턴, 구조, 관계 등을 파악하고 이해합니다. 이 과정에서 인공 신경망과 같은 딥러닝 기술이 핵심적으로 활용됩니다.
*   **프롬프트 기반:** 사용자가 텍스트, 이미지 등 특정 형태의 '프롬프트(명령어)'를 입력하면, AI는 이를 바탕으로 학습한 내용을 활용해 새로운 결과물을 생성합니다.
*   **파운데이션 모델:** GPT-3와 같은 대규모 언어 모델(LLM)과 Stable Diffusion 같은 이미지 생성 모델이 대표적인 예시입니다.

**활용 분야:**
생성형 AI는 글쓰기(시, 에세이, 코드), 이미지 및 비디오 제작, 음악 작곡, 번역, 요약 등 다양한 분야에서 활용됩니다. 마케팅, 제품 개발, 고객 서비스, 예술 등 여러 산업에서 창의적이고 혁신적인 콘텐츠 제작을 지원하며 활용 범위가 확장되고 있습니다.

👤 User :   
